##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [2]:
import tensorflow as tf

Import MLflow and enable autologging functionality.

In [3]:
import mlflow
mlflow.tensorflow.autolog()

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.13375668,  0.6109406 ,  0.01651868, -0.15387765,  0.26776463,
         0.25587657,  0.13770157, -0.49840438, -0.02513014, -0.36549386]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.10504528, 0.1692836 , 0.09342449, 0.07878773, 0.12010898,
        0.11868957, 0.10546049, 0.05582548, 0.08961339, 0.06376095]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [9]:
loss_fn(y_train[:1], predictions).numpy()

2.1312437

In [10]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [11]:
model.fit(x_train, y_train, epochs=5)

2021/05/26 16:39:32 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '2b9990fa0bef4b55b8af511f9fd4392c', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.4792 - accuracy: 0.8610
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1533 - accuracy: 0.9558
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1073 - accuracy: 0.9673
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0900 - accuracy: 0.9721
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0752 - accuracy: 0.9761


Using TensorFlow backend.


INFO:tensorflow:Assets written to: /tmp/tmp9qjef9a4/model/data/model/assets


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [12]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0722 - accuracy: 0.9775


[0.072201207280159, 0.9775000214576721]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [13]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.39459209e-08, 2.67430309e-08, 7.42814564e-06, 1.09223176e-04,
        7.29629412e-11, 4.58298608e-08, 6.12389303e-14, 9.99882460e-01,
        1.46080737e-07, 7.20122273e-07],
       [5.11577181e-09, 3.62628016e-05, 9.99953270e-01, 7.45221826e-07,
        3.39957669e-15, 1.14117768e-06, 6.10528472e-09, 1.17918121e-12,
        8.54884001e-06, 2.60742355e-13],
       [1.79647913e-07, 9.99542952e-01, 1.27839332e-04, 1.47128253e-06,
        1.25984470e-05, 9.04683020e-07, 3.68165820e-06, 2.86890339e-04,
        2.26824395e-05, 8.27269048e-07],
       [9.99743402e-01, 8.30657418e-11, 2.30656398e-04, 1.95882883e-08,
        3.50718850e-08, 3.52107668e-06, 3.67155053e-06, 1.16390120e-05,
        1.36343559e-08, 7.15067426e-06],
       [4.47857815e-07, 2.20714780e-09, 1.06071038e-05, 1.14239960e-07,
        9.98625398e-01, 2.66457300e-06, 3.34696324e-07, 2.05765828e-04,
        1.76843253e-06, 1.15290668e-03]], dtype=float32)>